In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import json
import time
import utils
from collections import defaultdict
from tqdm.notebook import tqdm, tqdm_notebook
tqdm_notebook.pandas()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
def sub_players(eventos, partidas):
    subs_match = {}

    for match, id in zip(partidas['teamsData'], partidas['wyId']):
        try:
            subs = {}
            for team in match.values():
                for changes in team['formation']['substitutions']:
                    subs[changes['playerIn']] = changes['playerOut']
            subs_match[id] = subs
        except TypeError:
            pass

    #transformar dicionário com matchId / playerIn / playerOut em data frame
    df_sub = pd.DataFrame(
        [[k1, k2, v]
        for k1,d in subs_match.items() 
        for k2,v in d.items()],
        columns=['matchId' , 'playerIn' , 'playerOut'])

    eventos = eventos.merge(df_sub, how='left', left_on=['matchId', 'playerId'], right_on=['matchId', 'playerIn']).drop(['playerIn'], axis=1)

    eventos['playerId'] = eventos['playerOut'].fillna(eventos['playerId'])
    eventos.drop(['playerOut'], axis = 1, inplace=True)

    return eventos


In [4]:
print("ALTERANDO JOGADORES SUBSTITUTOS")
for tournament in utils.TOURNAMENTS:
    print("Tournament : %s" %tournament)

    events = pd.read_json('data_backup\events\events_%s.json' %tournament)
    matches = pd.read_json('data_backup\matches\matches_%s.json' %tournament)

    df = sub_players(events, matches)
    df.to_json("events/events_%s.json" %tournament, orient='records')

ALTERANDO JOGADORES SUBSTITUTOS
Tournament : Italy
Tournament : England
Tournament : Germany
Tournament : France
Tournament : Spain
Tournament : European_Championship
Tournament : World_Cup
